<a href="https://colab.research.google.com/github/jovannip/stablepy/blob/main/%22SSD_Turbo_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Installing
import torch
gpu = torch.cuda.is_available()
!pip install diffusers transformers accelerate googletrans==3.1.0a0 numba xformers==0.0.22.post7 --upgrade
from IPython.display import clear_output
from diffusers import AutoPipelineForText2Image,AutoencoderKL
import torch

clear_output()

if gpu==True:
  pipe = AutoPipelineForText2Image.from_pretrained("kitty7779/ponyDiffusionV6XL", custom_pipeline="lpw_stable_diffusion_xl", torch_dtype=torch.float16, use_safetensors=True, add_watermarker=False)
  cust_vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
  pipe.vae = cust_vae
  pipe.to("cuda")
  pipe.enable_xformers_memory_efficient_attention()
  clear_output()
  # variant A
  !wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Concept%20Art%20Style%20SDXL_LoRA_Pony%20Diffusion%20V6%20XL.safetensors -P /content/lora
  !wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Smooth%20Anime%202%20Style%20SDXL_LoRA_Pony%20Diffusion%20V6%20XL.safetensors -P /content/lora
  print("GPU usage")
  print("OK!")
else:
  print("GPU was not find. Switching to CPU usage. The number of steps is automatically set to 1 for higher perfomance")
  pipe = AutoPipelineForText2Image.from_pretrained("AstraliteHeart/pony-diffusion-v6",weight_name="v6.safetensors")
  clear_output()
  print("CPU usage")
  print("OK!")

In [ ]:
# @title Download LORA

!wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Concept%20Art%20Style%20SDXL_LoRA_Pony%20Diffusion%20V6%20XL.safetensors -P /content/lora
!wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Smooth%20Anime%202%20Style%20SDXL_LoRA_Pony%20Diffusion%20V6%20XL.safetensors -P /content/lora
!wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Anime%20Summer%20Days%202%20Style%20SDXL_LoRA_Pony%20Diffusion%20V6%20XL.safetensors -P /content/lora
!wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Anime%20Summer%20Night%20Style%20SDXL_LoRA_Pony%20Diffusion%20V6%20XL.safetensors -P /content/lora
!wget https://huggingface.co/jiovannip/s17x1_/resolve/main/Line%20Art%20Style%20LoRA%20XL.safetensors -P /content/lora

In [ ]:

#@title Set DPM++ 2M SDE Karras Scheduler

from diffusers import DPMSolverMultistepScheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config,use_karras_sigmas=True, algorithm_type="sde-dpmsolver++")

In [ ]:

#@title Set EulerA scheduler

from diffusers import EulerAncestralDiscreteScheduler

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:


#@title Load Concept Art LORA

pipe.load_lora_weights("/content/lora/Concept Art Style SDXL_LoRA_Pony Diffusion V6 XL.safetensors", cross_attention_kwargs={"scale": 0.8})

In [ ]:

#@title Load Smooth Anime 2 LORA

pipe.load_lora_weights("/content/lora/Smooth Anime 2 Style SDXL_LoRA_Pony Diffusion V6 XL.safetensors", cross_attention_kwargs={"scale": 0.8})

In [ ]:
#@title Load Anime Summer Days 2 LORA

pipe.load_lora_weights("/content/lora/Anime Summer Days Style SDXL_LoRA_Pony Diffusion V6 XL.safetensors", cross_attention_kwargs={"scale": 0.8})

In [ ]:
#@title Load Anime Summer Night LORA

pipe.load_lora_weights("/content/lora/Anime Summer Night Style SDXL_LoRA_Pony Diffusion V6 XL.safetensors", cross_attention_kwargs={"scale": 0.8})

In [ ]:
#@title Load Line Art LORA

pipe.load_lora_weights("/content/lora/Line Art Style LoRA XL.safetensors", cross_attention_kwargs={"scale": 0.8})

In [ ]:
#@title Unload LORA

pipe.unload_lora_weights()

In [ ]:
# @title Generation
Prompt = "score_9, score_8_up, score_7_up, 1boy, fat boy, 1girl, slim figure, medium hair, brown hair, natural small breasts, hetero, cowgirl pose, slouching, blush, in heat, embarrassed, interrupted by viewer, lips parted, dutch angle, grabbed by hair, grab leg, implied penetration, panties around leg, navel, pubic hair, sweat, bedroom , shadowed, from behind, (highly detailed, foreshortening), realistic shadows" # @param {type:"string"}
negative_prompt= "score_4,score_5,score_6, simple background, censored, lowres, low detail, low-quality, signature, distorted, jpeg artifacts, compression artifacts, poorly drawn, low-resolution, bad, distortion, twisted, excessive, exaggerated pose,blurry, watercolor, thick thighs, muscular female, large breasts, bad anatomy, bad proportions, deformed, deformed anatomy, deformed fingers, running mascara, 3d" # @param {type:"string"}
Steps = 30 # @param {type:"number"}
width = 768
height= 1216
scale = 6.5

from googletrans import Translator, constants
from pprint import pprint
from google.colab import files
translator = Translator()
translation = translator.translate(Prompt)
PromptEN = translation.text
prompt = PromptEN
if gpu==False:
  image = pipe(prompt=PromptEN, num_inference_steps=1, guidance_scale=0.0).images[0]
else:
  image = pipe(prompt=PromptEN, negative_prompt=negative_prompt, width=width, height=height, num_inference_steps=Steps, clip_scale=2, guidance_scale=scale).images[0]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:

#@title Show result
display(image)

In [ ]:
#@title Clear CUDA cache
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()